In [95]:
import pandas as pd
import numpy as np
import random
import scipy.stats as stats
import yfinance as yf

from imblearn.over_sampling import SMOTE

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from imblearn.over_sampling import SMOTE


In [96]:
data = pd.read_excel('Newsraul.xlsx')

In [97]:
data['Label'].value_counts()

Label
1    100
0     99
Name: count, dtype: int64

In [122]:
data

,Date,Label,News,Combined_Text
0,2023-11-09,0,"Venda de cimento cai 2,1% em outubro sobre um ...","Venda de cimento cai 2,1% em outubro sobre um ..."
1,2023-11-09,0,Aï¿½ï¿½es da CBA (CBAV3) despencam com prejuï¿...,Aï¿½ï¿½es da CBA (CBAV3) despencam com prejuï¿...
2,2023-11-09,0,"Vieira diz que diretora entrega resultado, mas...","Vieira diz que diretora entrega resultado, mas..."
3,2023-11-09,1,"UBS emite US$ 3,5 bi em tï¿½tulos AT1, na prim...","UBS emite US$ 3,5 bi em tï¿½tulos AT1, na prim..."
4,2023-11-09,1,Aï¿½ï¿½es atingem maior nï¿½vel em trï¿½s sema...,Aï¿½ï¿½es atingem maior nï¿½vel em trï¿½s sema...
...,...,...,...,...
194,NaT,1,"Ouro fecha em alta, com juros dos Treasuries e...","Ouro fecha em alta, com juros dos Treasuries e..."
195,NaT,1,Construtoras do Reino Unido são favorecidas co...,Construtoras do Reino Unido são favorecidas co...
196,NaT,0,"Jalles registrou prejuízo de R$ 47,6 milhões n...","Jalles registrou prejuízo de R$ 47,6 milhões n..."
197,NaT,1,Abate bovinos no Brasil cresce 11% e é o maior...,Abate bovinos no Brasil cresce 11% e é o maior...



Naive Bayes

In [98]:
combined_data = data
# Preprocess the text data and create a sentiment label column
combined_data['Label'] = combined_data['Label'].apply(lambda x: 1 if x == 1 else 0)

# Convert all columns to string (except 'Label')
combined_data[combined_data.columns.difference(['Label'])] = combined_data[combined_data.columns.difference(['Label'])].astype(str)

# Combine the Top1 to Top25 columns
combined_data['Combined_Text'] = combined_data.iloc[:, 2:].apply(lambda row: ' '.join(row), axis=1)

# Define a custom tokenizer function to exclude words with length smaller than 4
def custom_tokenizer(text):
    words = re.findall(r'\b\w{4,}\b', text)
    return words

# Split the data into training and testing sets
X = combined_data['Combined_Text']
y = combined_data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer with the custom tokenizer
vectorizer = TfidfVectorizer(max_features=5000, tokenizer=custom_tokenizer, min_df=9)

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Predict sentiment labels on the testing data
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.60
              precision    recall  f1-score   support

           0       0.80      0.36      0.50        22
           1       0.53      0.89      0.67        18

    accuracy                           0.60        40
   macro avg       0.67      0.63      0.58        40
weighted avg       0.68      0.60      0.58        40



c:\Users\RaulR\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [99]:
Ticker = pd.read_excel('News_comp.xlsx')

In [100]:
Ticker = Ticker.dropna(axis=1, how='all')


In [101]:
Ticker

,PETR3,VALE3,ITUB4
0,Salto da Petrobras (PETR3;PETR4) “segura” Ibov...,Minério de ferro em Dalian vai à máxima de 2 a...,"Expectativas de inflação em 3,5% incomodam e d..."
1,CITI PROJETA LUCRO DE US$6 BI P/ PETROBRAS NO 3T,Vale usa energia eólica no maior navio de miné...,Mercado reage bem ao balanço do Itaú Unibanco ...
2,CITI REITERA RECOMENDAÇÃO DE 'COMPRA' PARA ADR...,Vale diz que incêndio paralisou transporte de ...,Itaú vê queda na inadimplência em pessoa físic...
3,CITI ELEVA PREÇO-ALVO DE ADR DA PETROBRAS DE U...,"Imposto seletivo poderia cortar em até R$1,16 ...","Itaú: Receitas com serviços somam R$ 10,694 bi..."
4,"CEO DA PETROBRAS, PRATES: DIESEL RUSSO DEIXOU ...",Minério de ferro tem sessão morna com cautela ...,Itaú Unibanco tem lucro líquido gerencial de R...
5,"ESTOQUES DE PETRÓLEO API VARIAM 11,90 MI NA SE...",Ação mira bancos nos EUA que financiaram barra...,Itaú (ITUB4): Níveis de rentabilidade atuais p...
6,"Lucro da Petrobras cai 42% e atinge R$ 26,6 bi...",Projeção no Congresso neste domingo lembra des...,"Itaú (ITUB4) sofre “efeito Argentina”, mas mos..."
7,Lucro líquido da Petrobras cai mais de 40% no ...,Minério de ferro registra novo ganho semanal c...,Itaú Unibanco tem lucro recorrente de R$9 bi n...
8,Petrobras eleva estimativa de produção de óleo...,Atividade industrial da China tem contração em...,"Itaú consegue retorno maior no varejo, e outlo..."
9,"Petrobras anuncia pagamento de R$17,5 bi em di...",Metal Leve (LEVE3) acelera queda em meio a ofe...,Itaú (ITUB4) Conclui Venda de Unidade na Argen...


In [102]:
# Create separate DataFrames for each ticker
df_petr3 = pd.DataFrame({'News': Ticker['PETR3']})
df_vale3 = pd.DataFrame({'News': Ticker['VALE3']})
df_itub4 = pd.DataFrame({'News': Ticker['ITUB4']})

# Add a 'Sentiment_Score' column for sentiment analysis
df_petr3['Sentiment_Score'] = 0
df_vale3['Sentiment_Score'] = 0
df_itub4['Sentiment_Score'] = 0

Training Model

In [103]:
def train_sentiment_model(data):
    # Copy the data 
    combined_data = data.copy()

    # Preprocess the text data and create a sentiment label column
    combined_data['Label'] = combined_data['Label'].apply(lambda x: 1 if x == 1 else 0)

    # Convert all columns to string (except 'Label')
    combined_data[combined_data.columns.difference(['Label'])] = combined_data[combined_data.columns.difference(['Label'])].astype(str)

    # Combine the Text
    combined_data['Combined_Text'] = combined_data.iloc[:, 2:].apply(lambda row: ' '.join(row), axis=1)

    # Tokenizer function to exclude words with length smaller than 4
    def custom_tokenizer(text):
        words = re.findall(r'\b\w{4,}\b', text)
        return words

    # Split the data into training and testing sets
    X = combined_data['Combined_Text']
    y = combined_data['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer(max_features=5000, tokenizer=custom_tokenizer, min_df=9)  

    # Fit and transform the training data
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train a Multinomial Naive Bayes model
    model = MultinomialNB()
    model.fit(X_train_tfidf, y_train)

    # Predict sentiment labels on the testing data
    y_pred = model.predict(X_test_tfidf)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return model, vectorizer

In [104]:
trained_model, trained_vectorizer = train_sentiment_model(data)

Accuracy: 0.60
              precision    recall  f1-score   support

           0       0.80      0.36      0.50        22
           1       0.53      0.89      0.67        18

    accuracy                           0.60        40
   macro avg       0.67      0.63      0.58        40
weighted avg       0.68      0.60      0.58        40



c:\Users\RaulR\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [105]:
def analyze_sentiment_on_dataframe(df, model, vectorizer):
    # Convert the entire DataFrame to strings
    df = df.astype(str)

    # Create a new column to store the sentiment scores
    df['Sentiment_Score'] = 0

    # Iterate through columns (tickers) and analyze sentiment
    for column in df.columns:
        if column != 'index': 
            # Combine the specified text columns
            combined_text = []
            for news in df[column]:
                if pd.notna(news):
                    if isinstance(news, str):
                        combined_text.append(' '.join(news.split()))
                    elif isinstance(news, int):  
                        combined_text.append(str(news))
                    else:
                        combined_text.append('')
                else:
                    combined_text.append('')

            df['Combined_Text'] = combined_text

            # Transform the data
            X_tfidf = vectorizer.transform(df['Combined_Text'])

            # Predict sentiment labels on the data
            y_pred = model.predict(X_tfidf)

            # Update Sentiment_Score based on the predicted labels
            df['Sentiment_Score'] += np.where(y_pred == 1,1,-1)

    return df


In [106]:
result_data_petr3 = analyze_sentiment_on_dataframe(df_petr3, trained_model, trained_vectorizer)
result_data_petr3['Combined_Text'].astype(int).sum() / 29

0.5862068965517241

In [107]:
result_data_vale3 = analyze_sentiment_on_dataframe(df_vale3, trained_model, trained_vectorizer)
result_data_vale3['Combined_Text'].astype(int).sum() / 29


0.7241379310344828

In [108]:
result_data_itub4 = analyze_sentiment_on_dataframe(df_itub4, trained_model, trained_vectorizer)
result_data_itub4['Combined_Text'].astype(int).sum() /29

0.9310344827586207

# Case Study

In [120]:
stock_prices_data = pd.read_csv('prices/PETR3_stock_data.csv')

# 'Close' in your stock_prices_data DataFrame
stock_prices = stock_prices_data['Close']

# 'Sentiment_Score'
sentiment_scores = result_data_petr3['Combined_Text'].astype(int).sample(10)

# Choose your significance level (alpha)
alpha = 0.05

# Perform Pearson correlation test
pearson_corr, pearson_p_value = stats.pearsonr(sentiment_scores, stock_prices)

# Perform Spearman correlation test
spearman_corr, spearman_p_value = stats.spearmanr(sentiment_scores, stock_prices)

# Print the results
print(f"Pearson Correlation: {pearson_corr}")
print(f"Pearson P-Value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}")
print(f"Spearman P-Value: {spearman_p_value}")

# To reject the null hypothesis
if pearson_p_value < alpha:
    print("Reject the null hypothesis for Pearson correlation. There is a significant correlation.")
else:
    print("Fail to reject the null hypothesis for Pearson correlation. No significant correlation.")

if spearman_p_value < alpha:
    print("Reject the null hypothesis for Spearman correlation. There is a significant correlation.")
else:
    print("Fail to reject the null hypothesis for Spearman correlation. No significant correlation.")

Pearson Correlation: 0.28970848671569993
Pearson P-Value: 0.4168278393741377
Spearman Correlation: 0.34188172937891387
Spearman P-Value: 0.33358700175210276
Fail to reject the null hypothesis for Pearson correlation. No significant correlation.
Fail to reject the null hypothesis for Spearman correlation. No significant correlation.


In [111]:
stock_prices

0    38.480000
1    38.169998
2    37.840000
3    38.279999
4    38.599998
5    38.500000
6    37.650002
7    36.759998
8    37.540001
9    37.419998
Name: Close, dtype: float64

In [121]:
sentiment_scores

18    1
3     1
23   -1
22    1
16    1
7     1
10   -1
19    1
25   -1
5     1
Name: Combined_Text, dtype: int32

In [115]:
print(len(sentiment_scores))
print(len(stock_prices))

10
10
